### Cài đặt môi trường

In [2]:
%%capture
import os
os.environ["UNSLOTH_VLLM_STANDBY"] = "1" # [NEW] Extra 30% context lengths!
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install or uv pip install
    !pip install unsloth vllm
else:
    pass # For Colab / Kaggle, we need extra instructions hidden below \/

In [3]:
%%capture
import os
!pip install --upgrade -qqq uv
if "COLAB_" not in "".join(os.environ.keys()):
    # If you're not in Colab, just use pip install!
    !pip install unsloth vllm
else:
    try: import numpy, PIL; get_numpy = f"numpy=={numpy.__version__}"; get_pil = f"pillow=={PIL.__version__}"
    except: get_numpy = "numpy"; get_pil = "pillow"
    try: import subprocess; is_t4 = "Tesla T4" in str(subprocess.check_output(["nvidia-smi"]))
    except: is_t4 = False
    get_vllm, get_triton = ("vllm==0.9.2", "triton==3.2.0") if is_t4 else ("vllm==0.10.2", "triton")
    !uv pip install -qqq --upgrade \
        unsloth {get_vllm} {get_numpy} {get_pil} torchvision bitsandbytes xformers
    !uv pip install -qqq {get_triton}
!uv pip install transformers==4.56.2
!uv pip install --no-deps trl==0.22.2

### Tải Model Vietnamese Legal Llama3.2-3B

Tải model `Vietnamese Legal Llama3.2-3B` đã được SFT và thiết lập các tham số LoRA tối ưu cho T4 15GB VRAM:

In [4]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1536 
lora_rank = 32 

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2",
    max_seq_length = max_seq_length,
    load_in_4bit = False, 
    fast_inference = True, 
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.85, 
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Optimized for T4 memory constraints
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank,
    use_gradient_checkpointing = "unsloth", 
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-12-02 11:22:32.711368: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1764674552.932966      47 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1764674552.994903      47 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


INFO 12-02 11:23:06 [__init__.py:244] Automatically detected platform cuda.
ERROR 12-02 11:23:08 [fa_utils.py:57] Cannot use FA version 2 is not supported due to FA2 is only supported on devices with compute capability >= 8
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 12-02 11:23:22 [vllm_utils.py:702] Unsloth: Patching vLLM v1 graph capture
INFO 12-02 11:23:22 [vllm_utils.py:732] Unsloth: Patching vLLM v0 graph capture
==((====))==  Unsloth 2025.11.6: Fast Llama patching. Transformers: 4.56.2. vLLM: 0.9.2.
   \\   /|    Tesla T4. Num GPUs = 2. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.7.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.30. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!
Unsloth: Standby mode is enabled. Changing `gpu_memory_utilization` to 0.85.
Unsloth: v

`torch_dtype` is deprecated! Use `dtype` instead!


INFO 12-02 11:23:41 [config.py:1472] Using max model len 1536
WARNING 12-02 11:23:41 [arg_utils.py:1735] Compute Capability < 8.0 is not supported by the V1 Engine. Falling back to V0. 
INFO 12-02 11:23:44 [config.py:2285] Chunked prefill is enabled with max_num_batched_tokens=4096.
INFO 12-02 11:23:44 [llm_engine.py:230] Initializing a V0 LLM engine (v0.9.2) with config: model='mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2', speculative_config=None, tokenizer='mikeethanh/vietnamese-legal-llama3.2-3b-merged-sft-v2', skip_tokenizer_init=False, tokenizer_mode=auto, revision=None, override_neuron_config={}, tokenizer_revision=None, trust_remote_code=False, dtype=torch.float16, max_seq_len=1536, download_dir=None, load_format=LoadFormat.AUTO, tensor_parallel_size=1, pipeline_parallel_size=1, disable_custom_all_reduce=False, quantization=None, enforce_eager=False, kv_cache_dtype=auto,  device_config=cuda, decoding_config=DecodingConfig(backend='auto', disable_fallback=False, disable

...

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json:   0%|          | 0.00/17.2M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/454 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/943 [00:00<?, ?B/s]

Unsloth 2025.11.6 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


### Chuẩn bị dữ liệu luật giao thông
<a name="Data"></a>

Chúng ta sẽ tải dataset câu hỏi-đáp về luật giao thông Việt Nam và định dạng cho GRPO training:

In [5]:
import json
from datasets import Dataset

# Load Vietnamese legal traffic law dataset
data_file = "/kaggle/input/synthetic-traffic-legal/synthetic_legal_qa_data.jsonl"

with open(data_file, "r", encoding="utf-8") as f:
    data = [json.loads(line) for line in f]

print(f"✅ Loaded {len(data)} Vietnamese legal Q&A pairs from {data_file}")
print("\n📋 Sample data:")
print(f"❓ Question: {data[0]['question']}")
print(f"💡 Answer: {data[0]['answer'][:150]}...")

# Show data statistics
avg_q_len = sum(len(item['question'].split()) for item in data) / len(data)
avg_a_len = sum(len(item['answer'].split()) for item in data) / len(data)
print(f"\n📊 Dataset statistics:")
print(f"   Average question length: {avg_q_len:.1f} words")
print(f"   Average answer length: {avg_a_len:.1f} words")

✅ Loaded 806 Vietnamese legal Q&A pairs from /kaggle/input/synthetic-traffic-legal/synthetic_legal_qa_data.jsonl

📋 Sample data:
❓ Question: Đường phố và đường bộ có gì khác nhau?
💡 Answer: Theo văn bản pháp luật đã cung cấp, "đường bộ" là khái niệm chung bao gồm tất cả các loại đường, cầu, hầm phục vụ cho phương tiện giao thông, trong kh...

📊 Dataset statistics:
   Average question length: 14.9 words
   Average answer length: 167.6 words


### Định dạng dữ liệu cho GRPO

In [6]:
# Define reasoning markers for structured legal responses
reasoning_start = "<start_working_out>"
reasoning_end   = "<end_working_out>"
solution_start = "<SOLUTION>"
solution_end = "</SOLUTION>"

# Updated system prompt for Vietnamese legal reasoning
system_prompt = f"""Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần {reasoning_start} {reasoning_end}, sau đó xuống 1 dòng, cách 1 dòng.
2. Đưa ra câu trả lời chính xác trong phần {solution_start}{solution_end}

Nguyên tắc hoạt động:
1. Cung cấp thông tin pháp lý chính xác theo quy định của pháp luật Việt Nam
2. Trả lời bằng tiếng Việt rõ ràng, dễ hiểu
3. Phân tích vấn đề một cách logic và có cấu trúc
4. Đưa ra lời khuyên thực tế và khả thi
5. Khi cần thiết, đề xuất tham khảo thêm chuyên gia pháp lý

Lưu ý: Thông tin này chỉ mang tính chất tham khảo. Để có lời khuyên chính thức, hãy tham khảo luật sư hoặc cơ quan có thẩm quyền."""

print(system_prompt)

Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:
1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out>, sau đó xuống 1 dòng, cách 1 dòng.
2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION>

Nguyên tắc hoạt động:
1. Cung cấp thông tin pháp lý chính xác theo quy định của pháp luật Việt Nam
2. Trả lời bằng tiếng Việt rõ ràng, dễ hiểu
3. Phân tích vấn đề một cách logic và có cấu trúc
4. Đưa ra lời khuyên thực tế và khả thi
5. Khi cần thiết, đề xuất tham khảo thêm chuyên gia pháp lý

Lưu ý: Thông tin này chỉ mang tính chất tham khảo. Để có lời khuyên chính thức, hãy tham khảo luật sư hoặc cơ quan có thẩm quyền.


In [7]:
# Function to extract answer and format for reasoning structure
def extract_structured_answer(answer_text):
    """Convert regular answer to structured format with reasoning"""
    # Simple approach: treat the answer as the solution
    # In practice, you might want to use an LLM to generate reasoning
    return answer_text.strip()

# Format data for GRPO training with reasoning structure
def format_legal_qa_reasoning(examples):
    prompts = []
    answers = []
    
    for question, answer in zip(examples["question"], examples["answer"]):
        # Create conversation format with reasoning prompt
        prompt = [
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": question}
        ]
        prompts.append(prompt)
        
        # For training, we expect the model to generate structured answers
        # The answer will be used to reward models that match this format
        structured_answer = extract_structured_answer(answer)
        answers.append(structured_answer)
    
    return {
        "prompt": prompts,
        "answer": answers
    }

# Convert to Hugging Face Dataset and format
dataset = Dataset.from_list(data)
dataset = dataset.map(format_legal_qa_reasoning, batched=True, remove_columns=dataset.column_names)
print(f"Formatted {len(dataset)} examples for GRPO training")
print("\nExample formatted data:")
print("Prompt:", dataset[0]["prompt"])
print("Answer:", dataset[0]["answer"][:200] + "...")

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Formatted 806 examples for GRPO training

Example formatted data:
Prompt: [{'content': 'Bạn là một trợ lý AI chuyên về luật giao thông Việt Nam. Khi trả lời câu hỏi, hãy:\n1. Suy nghĩ và phân tích câu hỏi trong phần <start_working_out> <end_working_out>, sau đó xuống 1 dòng, cách 1 dòng.\n2. Đưa ra câu trả lời chính xác trong phần <SOLUTION></SOLUTION>\n\nNguyên tắc hoạt động:\n1. Cung cấp thông tin pháp lý chính xác theo quy định của pháp luật Việt Nam\n2. Trả lời bằng tiếng Việt rõ ràng, dễ hiểu\n3. Phân tích vấn đề một cách logic và có cấu trúc\n4. Đưa ra lời khuyên thực tế và khả thi\n5. Khi cần thiết, đề xuất tham khảo thêm chuyên gia pháp lý\n\nLưu ý: Thông tin này chỉ mang tính chất tham khảo. Để có lời khuyên chính thức, hãy tham khảo luật sư hoặc cơ quan có thẩm quyền.', 'role': 'system'}, {'content': 'Đường phố và đường bộ có gì khác nhau?', 'role': 'user'}]
Answer: Theo văn bản pháp luật đã cung cấp, "đường bộ" là khái niệm chung bao gồm tất cả các loại đường, cầu, hầm phục 

In [8]:
# Regex pattern to match the structured reasoning format
import re

match_format = re.compile(
    rf"^[\s]{{0,}}"
    rf"{re.escape(reasoning_start)}.+?{re.escape(reasoning_end)}.*?"
    rf"{re.escape(solution_start)}(.+?){re.escape(solution_end)}"
    rf"[\s]{{0,}}$",
    flags = re.MULTILINE | re.DOTALL
)

# Test the regex pattern
test_response = f"{reasoning_start}Để trả lời câu hỏi này, tôi cần xem xét các quy định về luật giao thông...{reasoning_end}{solution_start}Theo Luật Giao thông đường bộ, mức phạt cho vi phạm này là 1-3 triệu đồng.{solution_end}"

match_result = match_format.search(test_response)
if match_result:
    print("✓ Regex pattern works correctly")
    print(f"Extracted solution: {match_result.group(1)}")
else:
    print("✗ Regex pattern needs adjustment")

✓ Regex pattern works correctly
Extracted solution: Theo Luật Giao thông đường bộ, mức phạt cho vi phạm này là 1-3 triệu đồng.


### Reward Functions cho Vietnamese Legal QA

In [9]:
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        # Match if format is seen exactly!
        if match_format.search(response) is not None: score += 3.0
        scores.append(score)
    return scores

def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0]["content"]
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
        score += 0.5 if response.count(solution_start)  == 1 else -1.0
        score += 0.5 if response.count(solution_end)    == 1 else -1.0
        scores.append(score)
    return scores

def reward_solution_length(completions, **kwargs):
    """
    Reward function để khuyến khích phần SOLUTION dài hơn (khoảng 400 từ)
    Không ảnh hưởng đến độ dài phần reasoning (working_out)
    """
    scores = []
    target_solution_words = 400  # Mục tiêu 400 từ cho phần SOLUTION
    min_solution_words = 200     # Tối thiểu 200 từ
    
    for completion in completions:
        score = 0.0
        response = completion[0]["content"]
        
        # Extract solution part using regex
        solution_pattern = rf"{re.escape(solution_start)}(.*?){re.escape(solution_end)}"
        solution_match = re.search(solution_pattern, response, re.DOTALL)
        
        if solution_match:
            solution_text = solution_match.group(1).strip()
            # Count Vietnamese words (split by spaces and common punctuation)
            solution_words = len(re.findall(r'\b\w+\b', solution_text))
            
            if solution_words >= 100 and solution_words < 200:
                # Bonus for 100-200 words range
                score += 1
            elif solution_words < 100:
                # Penalty for too short solutions (less than 200 words)
                score = -2.0
            elif solution_words >= min_solution_words and solution_words <= target_solution_words:
                score += 3
            else:
                # Slight penalty for being too long (over 400 words)
                excess = solution_words - target_solution_words
                score = 4.0 - min(excess / 200, 2.0)  # Gradual decrease
            
        else:
            # No solution section found
            score = -3.0
            
        scores.append(score)
    
    return scores

### Kiểm tra độ dài prompt tối đa

In [10]:
max_prompt_length = max(dataset.map(
    lambda x: {"tokens" : tokenizer.apply_chat_template(x["prompt"], add_generation_prompt = True, tokenize = True)},
    batched = True,
).map(lambda x: {"length" : len(x["tokens"])})["length"])

print(f"Maximum prompt length: {max_prompt_length} tokens")
max_prompt_length = min(max_prompt_length + 10, 400)  # Cap at 400 for T4 memory efficiency
print(f"Using max_prompt_length: {max_prompt_length} tokens")

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Map:   0%|          | 0/806 [00:00<?, ? examples/s]

Maximum prompt length: 235 tokens
Using max_prompt_length: 245 tokens


In [11]:
# Kiểm tra chi tiết từng reward function với nhiều test cases
def test_reward_functions_comprehensive():
    """Kiểm tra toàn diện các reward functions với nhiều test cases khác nhau"""
    
    # Test cases with different quality levels - cập nhật theo logic mới
    test_cases = [
        {
            "name": "Perfect Format + Target Solution (400 words)",
            "response": f"""{reasoning_start}Để trả lời câu hỏi này, tôi cần xem xét quy định về tốc độ trong khu vực đông dân cư theo Luật Giao thông đường bộ. Căn cứ vào Nghị định 100/2019/NĐ-CP, tốc độ tối đa được quy định rõ ràng.{reasoning_end}{solution_start}Theo quy định tại Điều 11 Luật Giao thông đường bộ và Nghị định 100/2019/NĐ-CP, tốc độ tối đa cho phép trong khu vực đông dân cư là 50 km/h đối với ô tô con và 40 km/h đối với xe tải. Quy định này được áp dụng nhằm đảm bảo an toàn giao thông trong các khu vực có mật độ dân cư cao, nơi có nhiều người đi bộ, xe đạp và các phương tiện giao thông khác tham gia. Việc tuân thủ giới hạn tốc độ này không chỉ giúp giảm thiểu nguy cơ tai nạn mà còn giảm mức độ nghiêm trọng khi tai nạn xảy ra. Người điều khiển phương tiện vi phạm quy định về tốc độ sẽ bị xử phạt hành chính theo mức độ vi phạm. Đối với việc vượt quá tốc độ cho phép từ 5-10 km/h, mức phạt từ 600.000 đến 1.000.000 đồng. Nếu vượt từ 10-20 km/h, mức phạt từ 1.200.000 đến 2.000.000 đồng. Trường hợp vượt quá 20 km/h trở lên sẽ bị phạt từ 6.000.000 đến 8.000.000 đồng và có thể bị tước giấy phép lái xe từ 2-4 tháng. Ngoài ra, việc lắp đặt và sử dụng thiết bị giám sát tốc độ cũng được quy định chi tiết để hỗ trợ công tác kiểm soát và xử lý vi phạm. Các biển báo giới hạn tốc độ phải được đặt ở vị trí dễ nhìn thấy và tuân thủ các tiêu chuẩn kỹ thuật quốc gia về an toàn giao thông đường bộ. Tất cả các quy định này đều nhằm mục tiêu cao nhất là bảo vệ an toàn của người dân.{solution_end}""",
            "expected_high": True
        },
        {
            "name": "Perfect Format + Minimum Range (200-250 words)",
            "response": f"""{reasoning_start}Cần xem xét quy định về tốc độ theo luật hiện hành.{reasoning_end}{solution_start}Theo Luật Giao thông đường bộ và Nghị định 100/2019/NĐ-CP, tốc độ tối đa trong khu vực đông dân cư là 50 km/h cho xe con và 40 km/h cho xe tải. Quy định này nhằm đảm bảo an toàn giao thông trong các khu vực có mật độ dân cư cao, nơi có nhiều người đi bộ và xe đạp tham gia. Vi phạm tốc độ sẽ bị xử phạt hành chính tùy theo mức độ vi phạm, từ phạt tiền đến tước giấy phép lái xe. Mức phạt cụ thể được quy định trong Nghị định về xử phạt vi phạm hành chính trong lĩnh vực giao thông đường bộ. Đối với việc vượt quá tốc độ cho phép từ 5-10 km/h, mức phạt từ 600.000 đến 1.000.000 đồng. Trường hợp vượt từ 10-20 km/h, mức phạt từ 1.200.000 đến 2.000.000 đồng. Vi phạm nghiêm trọng hơn sẽ bị phạt cao hơn và có thể bị tước giấy phép lái xe.{solution_end}""",
            "expected_high": True
        },
        {
            "name": "Perfect Format + Bonus Range (100-199 words)",
            "response": f"""{reasoning_start}Theo luật giao thông hiện hành{reasoning_end}{solution_start}Tốc độ tối đa trong khu dân cư là 50 km/h cho xe con, 40 km/h cho xe tải theo Luật Giao thông đường bộ. Quy định này được áp dụng để đảm bảo an toàn cho người dân trong khu vực có mật độ dân cư cao. Vi phạm tốc độ sẽ bị xử phạt theo mức độ nghiêm trọng của vi phạm. Mức phạt từ 600.000 đến 8.000.000 đồng tùy theo tốc độ vượt quá cho phép. Trường hợp nghiêm trọng có thể bị tước giấy phép lái xe từ 2-4 tháng. Các biển báo tốc độ phải được đặt rõ ràng để người điều khiển phương tiện dễ nhận biết và tuân thủ quy định.{solution_end}""",
            "expected_high": False
        },
        {
            "name": "Perfect Format + Very Short Solution (< 100 words)",
            "response": f"""{reasoning_start}Theo luật giao thông hiện hành{reasoning_end}{solution_start}Tốc độ tối đa trong khu dân cư là 50 km/h cho xe con, 40 km/h cho xe tải. Quy định này nhằm đảm bảo an toàn giao thông.{solution_end}""",
            "expected_high": False
        },
        {
            "name": "Perfect Format + Too Long Solution (> 400 words)",
            "response": f"""{reasoning_start}Cần phân tích chi tiết về quy định tốc độ{reasoning_end}{solution_start}Theo quy định tại Điều 11 Luật Giao thông đường bộ và Nghị định 100/2019/NĐ-CP, tốc độ tối đa cho phép trong khu vực đông dân cư là 50 km/h đối với ô tô con và 40 km/h đối với xe tải. Quy định này được áp dụng nhằm đảm bảo an toàn giao thông trong các khu vực có mật độ dân cư cao. Việc tuân thủ giới hạn tốc độ này không chỉ giúp giảm thiểu nguy cơ tai nạn mà còn giảm mức độ nghiêm trọng khi tai nạn xảy ra. Người điều khiển phương tiện vi phạm quy định về tốc độ sẽ bị xử phạt hành chính theo mức độ vi phạm. Đối với việc vượt quá tốc độ cho phép từ 5-10 km/h, mức phạt từ 600.000 đến 1.000.000 đồng. Nếu vượt từ 10-20 km/h, mức phạt từ 1.200.000 đến 2.000.000 đồng. Trường hợp vượt quá 20 km/h trở lên sẽ bị phạt từ 6.000.000 đến 8.000.000 đồng. Ngoài ra, việc lắp đặt và sử dụng thiết bị giám sát tốc độ cũng được quy định chi tiết. Các biển báo giới hạn tốc độ phải được đặt ở vị trí dễ nhìn thấy. Tất cả các quy định này đều nhằm mục tiêu cao nhất là bảo vệ an toàn của người dân. Đồng thời, các cơ quan chức năng cũng có trách nhiệm tuyên truyền giáo dục ý thức chấp hành pháp luật cho người dân. Việc thực hiện các biện pháp này sẽ góp phần giảm thiểu tai nạn giao thông và tạo môi trường giao thông an toàn cho mọi người.{solution_end}""",
            "expected_high": False
        },
        {
            "name": "No Format - Good Content",
            "response": "Theo quy định của pháp luật Việt Nam, tốc độ tối đa trong khu vực đông dân cư là 50 km/h cho xe con và 40 km/h cho xe tải. Điều này được quy định trong Luật Giao thông đường bộ.",
            "expected_high": False
        },
        {
            "name": "No Solution Section",
            "response": f"""{reasoning_start}Chỉ có phần suy nghĩ mà không có solution{reasoning_end}""",
            "expected_high": False
        }
    ]
    
    print("🔍 KIỂM TRA CHI TIẾT 3 REWARD FUNCTIONS")
    print("=" * 80)
    print("🎯 Focus: Format + Solution Length (Updated Logic)")
    print("📏 New Scoring System:")
    print("   - 100-199 words: +1 point (bonus)")
    print("   - <100 words: -2 points (penalty)")  
    print("   - 200-400 words: +3 points (optimal)")
    print("   - >400 words: decreasing penalty")
    print("-" * 80)
    
    # Missing functions - thêm vào để test hoàn chỉnh
    def match_format_exactly(completions, **kwargs):
        scores = []
        for completion in completions:
            score = 0
            response = completion[0]["content"]
            if match_format.search(response) is not None: score += 3.0
            scores.append(score)
        return scores

    def match_format_approximately(completions, **kwargs):
        scores = []
        for completion in completions:
            score = 0
            response = completion[0]["content"]
            score += 0.5 if response.count(reasoning_start) == 1 else -1.0
            score += 0.5 if response.count(reasoning_end)   == 1 else -1.0
            score += 0.5 if response.count(solution_start)  == 1 else -1.0
            score += 0.5 if response.count(solution_end)    == 1 else -1.0
            scores.append(score)
        return scores
    
    # 3 reward functions
    reward_functions = [
        ("Format Exactly", match_format_exactly),
        ("Format Approximately", match_format_approximately),
        ("Solution Length", reward_solution_length),
    ]
    
    for test_case in test_cases:
        print(f"\n📝 Test Case: {test_case['name']}")
        print(f"Expected Quality: {'HIGH' if test_case['expected_high'] else 'LOW'}")
        
        # Count words in solution section
        solution_pattern = rf"{re.escape(solution_start)}(.*?){re.escape(solution_end)}"
        solution_match = re.search(solution_pattern, test_case['response'], re.DOTALL)
        if solution_match:
            solution_words = len(re.findall(r'\b\w+\b', solution_match.group(1).strip()))
            print(f"Solution words: {solution_words}")
            
            # Show expected score for this word count
            if solution_words >= 100 and solution_words < 200:
                print(f"Expected Length Score: +1 (bonus range)")
            elif solution_words < 100:
                print(f"Expected Length Score: -2 (too short)")
            elif solution_words >= 200 and solution_words <= 400:
                print(f"Expected Length Score: +3 (optimal range)")
            else:
                excess = solution_words - 400
                expected_score = 4.0 - min(excess / 200, 2.0)
                print(f"Expected Length Score: ~{expected_score:.1f} (too long penalty)")
        else:
            print("Solution words: 0 (no solution section)")
            print("Expected Length Score: -3 (no solution)")
        
        print(f"Response preview: {test_case['response'][:100]}{'...' if len(test_case['response']) > 100 else ''}")
        print("-" * 40)
        
        # Prepare test data
        test_completion = [[{"content": test_case['response']}]]
        
        total_score = 0
        for func_name, func in reward_functions:
            try:
                score = func(test_completion)[0]
                total_score += score
                print(f"  {func_name}: {score:.2f}")
            except Exception as e:
                print(f"  {func_name}: ERROR - {str(e)[:50]}...")
        
        print(f"  🎯 TOTAL SCORE: {total_score:.2f}")
        
        # Validate expectations - điều chỉnh threshold cho logic mới
        expected_threshold = 4.0  # Format (3.0 + 2.0) + Length (+1 to +3)
        if test_case['expected_high'] and total_score < expected_threshold:
            print("  ⚠️  WARNING: Expected high score but got low!")
        elif not test_case['expected_high'] and total_score >= expected_threshold:
            print("  ⚠️  WARNING: Expected low score but got high!")
        else:
            print("  ✅ Score aligns with expectations")
    
    print("\n" + "=" * 80)
    print("🎉 Updated reward testing completed!")
    
    # Test solution length specifically with new logic
    print("\n🧪 DETAILED SOLUTION LENGTH TESTING (New Logic)")
    print("=" * 60)
    
    length_tests = [
        ("Very Short (50 words)", f"{solution_start}" + " ".join(["từ"] * 50) + f"{solution_end}", -2.0),
        ("Short (80 words)", f"{solution_start}" + " ".join(["từ"] * 80) + f"{solution_end}", -2.0),
        ("Bonus Start (100 words)", f"{solution_start}" + " ".join(["từ"] * 100) + f"{solution_end}", 1.0),
        ("Bonus Mid (150 words)", f"{solution_start}" + " ".join(["từ"] * 150) + f"{solution_end}", 1.0),
        ("Bonus End (199 words)", f"{solution_start}" + " ".join(["từ"] * 199) + f"{solution_end}", 1.0),
        ("Optimal Start (200 words)", f"{solution_start}" + " ".join(["từ"] * 200) + f"{solution_end}", 3.0),
        ("Optimal Mid (300 words)", f"{solution_start}" + " ".join(["từ"] * 300) + f"{solution_end}", 3.0),
        ("Optimal End (400 words)", f"{solution_start}" + " ".join(["từ"] * 400) + f"{solution_end}", 3.0),
        ("Too Long (500 words)", f"{solution_start}" + " ".join(["từ"] * 500) + f"{solution_end}", "~3.5"),
        ("Very Long (600 words)", f"{solution_start}" + " ".join(["từ"] * 600) + f"{solution_end}", "~3.0"),
    ]
    
    for length_name, test_response, expected_score in length_tests:
        print(f"\n📋 Testing {length_name}:")
        test_completion = [[{"content": test_response}]]
        
        actual_score = reward_solution_length(test_completion)[0]
        print(f"   Expected Score: {expected_score}")
        print(f"   Actual Score: {actual_score:.1f}")
        
        # Validate if score matches expectation
        if isinstance(expected_score, float):
            if abs(actual_score - expected_score) < 0.1:
                print(f"   ✅ Score matches expectation")
            else:
                print(f"   ❌ Score mismatch!")
        else:
            print(f"   📝 Variable score (check manually)")
    
    print("\n" + "=" * 60)
    print("✅ Detailed solution length testing completed!")
    return True

# Run comprehensive testing
test_reward_functions_comprehensive()

🔍 KIỂM TRA CHI TIẾT 3 REWARD FUNCTIONS
🎯 Focus: Format + Solution Length (Updated Logic)
📏 New Scoring System:
   - 100-199 words: +1 point (bonus)
   - <100 words: -2 points (penalty)
   - 200-400 words: +3 points (optimal)
   - >400 words: decreasing penalty
--------------------------------------------------------------------------------

📝 Test Case: Perfect Format + Target Solution (400 words)
Expected Quality: HIGH
Solution words: 300
Expected Length Score: +3 (optimal range)
Response preview: <start_working_out>Để trả lời câu hỏi này, tôi cần xem xét quy định về tốc độ trong khu vực đông dân...
----------------------------------------
  Format Exactly: 3.00
  Format Approximately: 2.00
  Solution Length: 3.00
  🎯 TOTAL SCORE: 8.00
  ✅ Score aligns with expectations

📝 Test Case: Perfect Format + Minimum Range (200-250 words)
Expected Quality: HIGH
Solution words: 179
Expected Length Score: +1 (bonus range)
Response preview: <start_working_out>Cần xem xét quy định về tốc độ theo l

True

### Cấu hình GRPO Training tối ưu cho T4 15GB

In [12]:
from trl import GRPOConfig, GRPOTrainer

training_args = GRPOConfig(
    learning_rate = 6e-6,  # Lower LR for stability with legal content
    weight_decay = 0.05,   # Reduced weight decay
    warmup_ratio = 0.1,
    lr_scheduler_type = "cosine",
    optim = "adamw_8bit",  # Memory efficient optimizer
    logging_steps = 1,
    per_device_train_batch_size = 1,  # T4 memory constraint
    gradient_accumulation_steps = 8,  # Increased for effective batch size of 8
    num_generations = 4,   # Reduced for T4 memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_seq_length - max_prompt_length,
    num_train_epochs = 2,
    save_steps = 100,     # More frequent saves
    max_grad_norm = 1.0,
    report_to = "none",
    output_dir = "outputs",
    seed = 42
)

print("Training configuration optimized for T4 15GB VRAM")
print(f"Effective batch size: {training_args.per_device_train_batch_size * training_args.gradient_accumulation_steps}")
print(f"Max sequence length: {max_seq_length}")
print(f"Max prompt length: {training_args.max_prompt_length}")
print(f"Max completion length: {training_args.max_completion_length}")

Training configuration optimized for T4 15GB VRAM
Effective batch size: 8
Max sequence length: 1536
Max prompt length: 245
Max completion length: 1291


In [13]:
trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,              
        match_format_approximately,
        reward_solution_length,
    ],
    args = training_args,
    train_dataset = dataset,
)

print("🚀 Vietnamese Legal GRPO Trainer initialized!")
print("📊 Training metrics to watch:")
print("  - reward: Should increase as model learns Vietnamese legal reasoning")
print("  - reward_std: Standard deviation of rewards")
print("  - completion_length: Average Vietnamese response length")

print("  - kl: KL divergence (should stay reasonable < 1.0)")
print("="*80)


print("🎯 Goal: Train model to follow reasoning format consistently")
print("📝 Primary focus: <start_working_out>reasoning<end_working_out><SOLUTION>answer</SOLUTION>")
print("⚡ Reward function weights:")
print("  - Format Exactly: 3.0x (binary scoring for perfect format)")
print("  - Format Approximately: 1.5x (gradual learning for each component)")
print("  - Response Length: 2.0x (appropriate length for Vietnamese legal text)")

🚀 Vietnamese Legal GRPO Trainer initialized!
📊 Training metrics to watch:
  - reward: Should increase as model learns Vietnamese legal reasoning
  - reward_std: Standard deviation of rewards
  - completion_length: Average Vietnamese response length
  - kl: KL divergence (should stay reasonable < 1.0)
🎯 Goal: Train model to follow reasoning format consistently
📝 Primary focus: <start_working_out>reasoning<end_working_out><SOLUTION>answer</SOLUTION>
⚡ Reward function weights:
  - Format Exactly: 3.0x (binary scoring for perfect format)
  - Format Approximately: 1.5x (gradual learning for each component)
  - Response Length: 2.0x (appropriate length for Vietnamese legal text)


In [14]:
# Start training
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 806 | Num Epochs = 2 | Total steps = 806
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 8
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 8 x 1) = 8
 "-____-"     Trainable parameters = 48,627,712 of 3,261,377,536 (1.49% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss,reward,reward_std,completions / mean_length,completions / min_length,completions / max_length,completions / clipped_ratio,completions / mean_terminated_length,completions / min_terminated_length,completions / max_terminated_length,kl,rewards / match_format_exactly / mean,rewards / match_format_exactly / std,rewards / match_format_approximately / mean,rewards / match_format_approximately / std,rewards / reward_solution_length / mean,rewards / reward_solution_length / std
1,0.000000,3.375000,0.750000,255.125000,193.000000,311.000000,0.000000,255.125000,193.000000,311.000000,0.000008,3.000000,0.000000,2.000000,0.000000,-1.625000,1.060660
2,0.000000,2.500000,2.049038,253.125000,202.000000,309.000000,0.000000,253.125000,202.000000,309.000000,0.000007,2.250000,1.388730,1.625000,0.694365,-1.375000,1.505941
3,0.000000,2.812500,0.375000,247.750000,205.000000,305.000000,0.000000,247.750000,205.000000,305.000000,0.000007,3.000000,0.000000,1.812500,0.530330,-2.000000,0.000000
4,0.000000,3.750000,1.500000,264.375000,199.000000,341.000000,0.000000,264.375000,199.000000,341.000000,0.000006,3.000000,0.000000,2.000000,0.000000,-1.250000,1.388730
5,0.000000,2.687500,1.772181,278.375000,225.000000,350.000000,0.000000,278.375000,225.000000,350.000000,0.000007,2.625000,1.060660,1.812500,0.530330,-1.750000,1.164965
6,0.000000,2.812500,1.546165,264.875000,227.000000,321.000000,0.000000,264.875000,227.000000,321.000000,0.000007,2.625000,1.060660,1.812500,0.530330,-1.625000,1.060660
7,0.000000,3.750000,1.500000,261.250000,206.000000,352.000000,0.000000,261.250000,206.000000,352.000000,0.000006,3.000000,0.000000,2.000000,0.000000,-1.250000,1.388730
8,0.000000,3.750000,1.500000,272.750000,235.000000,319.000000,0.000000,272.750000,235.000000,319.000000,0.000006,3.000000,0.000000,2.000000,0.000000,-1.250000,1.388730
9,0.000000,1.375000,1.198958,260.125000,206.000000,334.000000,0.000000,260.125000,206.000000,334.000000,0.000007,1.875000,1.552648,1.625000,0.694365,-2.125000,0.353553
10,0.000000,4.125000,1.616025,258.500000,212.000000,302.000000,0.000000,258.500000,212.000000,302.000000,0.000006,3.000000,0.000000,2.000000,0.000000,-0.875000,1.552648


TrainOutput(global_step=806, training_loss=4.43373404769164e-05, metrics={'train_runtime': 35869.5206, 'train_samples_per_second': 0.045, 'train_steps_per_second': 0.022, 'total_flos': 0.0, 'train_loss': 4.43373404769164e-05})

### Lưu và test model đã được GRPO training

### Kiểm tra LoRA đã được training

Verify LoRA is actually trained!

Now we load the LoRA and test:

Our reasoning model is much better - it's not always correct, since we only trained it for an hour or so - it'll be better if we extend the sequence length and train for longer!

### Lưu Model cuối cùng
<a name="Save"></a>

Lưu model dưới nhiều định dạng khác nhau để sử dụng trong production:

In [20]:
# Lưu model dưới các định dạng khác nhau

# 1. Lưu LoRA adapters
model.save_pretrained_merged("vietnamese_legal_merged", tokenizer, save_method="merged_16bit")
print("✅ Merged model saved to: vietnamese_legal_merged/")


Found HuggingFace hub cache directory: /root/.cache/huggingface/hub
Checking cache directory for required files...





Unsloth: Copying 2 files from cache to `vietnamese_legal_merged`:   0%|          | 0/2 [00:00<?, ?it/s]


Unsloth: Copying 2 files from cache to `vietnamese_legal_merged`:  50%|█████     | 1/2 [00:10<00:10, 10.49s/it]


Unsloth: Copying 2 files from cache to `vietnamese_legal_merged`: 100%|██████████| 2/2 [00:15<00:00,  7.88s/it]


Successfully copied all 2 files from cache to `vietnamese_legal_merged`
Checking cache directory for required files...
Cache check failed: tokenizer.model not found in local cache.
Not all required files found in cache. Will proceed with downloading.





Unsloth: Preparing safetensor model files: 100%|██████████| 2/2 [00:00<00:00, 13400.33it/s]



Unsloth: Merging weights into 16bit:   0%|          | 0/2 [00:00<?, ?it/s]


Unsloth: Merging weights into 16bit:  50%|█████     | 1/2 [00:39<00:39, 39.19s/it]


Unsloth: Merging weights into 16bit: 100%|██████████| 2/2 [00:52<00:00, 26.33s/it]


Unsloth: Merge process complete. Saved to `/kaggle/working/vietnamese_legal_merged`
✅ Merged model saved to: vietnamese_legal_merged/


### Xuất GGUF cho llama.cpp (tùy chọn)

Nếu muốn sử dụng với llama.cpp hoặc Ollama:

In [23]:
# Upload merged model to HuggingFace Hub (giải pháp cho file lớn!)
# Bước 1: Tạo HuggingFace account tại https://huggingface.co/join
# Bước 2: Tạo token tại https://huggingface.co/settings/tokens (chọn "Write" permission)
# Bước 3: Thêm token vào Kaggle Secrets với key "HF_TOKEN"

import os

if os.path.exists("/kaggle/working"):
    print("🚀 Uploading model to HuggingFace Hub...")
    print("="*70)
    
    try:
        from kaggle_secrets import UserSecretsClient
        user_secrets = UserSecretsClient()
        hf_token = user_secrets.get_secret("HF_TOKEN")
        
        from huggingface_hub import HfApi, login
        
        # Login to HuggingFace
        login(token=hf_token)
        print("✅ Logged in to HuggingFace")
        
        # Thay YOUR_USERNAME bằng username HuggingFace của bạn
        YOUR_HF_USERNAME = "mikeethanh"  # ⚠️ SỬA DÒNG NÀY!
        repo_name = f"{YOUR_HF_USERNAME}/vietnamese-legal-llama3.2-3b-merged-grpo-v1"
        
        print(f"\n📤 Uploading to: {repo_name}")
        print("⏳ Đang upload ~6GB, có thể mất 10-15 phút...\n")
        
        # Upload merged model
        if os.path.exists("vietnamese_legal_merged"):
            from huggingface_hub import create_repo, upload_folder
            
            # Create repo (public)
            try:
                create_repo(repo_name, repo_type="model", exist_ok=True)
                print(f"✅ Repository created: https://huggingface.co/{repo_name}")
            except:
                print(f"ℹ️ Repository already exists: https://huggingface.co/{repo_name}")
            
            # Upload folder
            upload_folder(
                folder_path="vietnamese_legal_merged",
                repo_id=repo_name,
                commit_message="Vietnamese Legal AI - Llama 3.2 3B Merged Model",
            )
            
            print("\n" + "="*70)
            print("✅ UPLOAD THÀNH CÔNG!")
            print("="*70)
            print(f"\n📥 Download model về máy bằng cách:")
            print(f"   git clone https://huggingface.co/{repo_name}")
            print(f"\n🌐 Hoặc xem trên web:")
            print(f"   https://huggingface.co/{repo_name}")
            print("\n💡 Model đã public, ai cũng có thể download!")
        else:
            print("⚠️ Folder 'vietnamese_legal_merged' not found!")
            
    except Exception as e:
        print(f"❌ Error: {e}")
        print("\n📝 Hướng dẫn fix:")
        print("  1. Tạo account tại: https://huggingface.co/join")
        print("  2. Tạo token tại: https://huggingface.co/settings/tokens")
        print("  3. Kaggle: Add-ons → Secrets → Add 'HF_TOKEN'")
        print("  4. Sửa YOUR_USERNAME trong code")
        
else:
    print("ℹ️ This cell only works on Kaggle")
    print("💡 For local, use: model.push_to_hub() directly")

🚀 Uploading model to HuggingFace Hub...
✅ Logged in to HuggingFace

📤 Uploading to: mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1
⏳ Đang upload ~6GB, có thể mất 10-15 phút...

✅ Repository created: https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1


Processing Files (0 / 0): |          |  0.00B /  0.00B            

New Data Upload: |          |  0.00B /  0.00B            


✅ UPLOAD THÀNH CÔNG!

📥 Download model về máy bằng cách:
   git clone https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1

🌐 Hoặc xem trên web:
   https://huggingface.co/mikeethanh/vietnamese-legal-llama3.2-3b-merged-grpo-v1

💡 Model đã public, ai cũng có thể download!
